In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [3]:
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [11]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model="llama3.2", 
        messages=messages_for(website)
    )
    return response

In [13]:
context = summarize("https://www.aljazeera.com/")

In [14]:
context

ChatResponse(model='llama3.2', created_at='2024-11-22T11:14:20.112727438Z', done=True, done_reason='stop', total_duration=194526723129, load_duration=3625697982, prompt_eval_count=1549, prompt_eval_duration=148940000000, eval_count=218, eval_duration=40751000000, message=Message(role='assistant', content="**Summary of Al Jazeera Website**\n\n* **News and Announcements**\n\t+ Israeli air attack on Lebanon kills paramedics; ICC warrants issued for Netanyahu and Gallant\n\t+ Trump picks loyalist Pam Bondi as attorney general, Matt Gaetz withdraws\n\t+ Iran says activating 'advanced' centrifuges after IAEA censure\n\t+ At least 42 killed in sectarian violence in Pakistan's Khyber Pakhtunkhwa\n* **Features**\n\t+ Ukraine war: Mapping the impact of climate change on global displacement\n\t+ Bollywood is turning to re-releases amid string of flops\n\t+ World reacts to ICC arrest warrants for Israel's Netanyahu, Gallant\n* **Video**\n\t+ Putin: Russia tested intermediate-range missile on Ukrai

In [15]:
context.message.content

"**Summary of Al Jazeera Website**\n\n* **News and Announcements**\n\t+ Israeli air attack on Lebanon kills paramedics; ICC warrants issued for Netanyahu and Gallant\n\t+ Trump picks loyalist Pam Bondi as attorney general, Matt Gaetz withdraws\n\t+ Iran says activating 'advanced' centrifuges after IAEA censure\n\t+ At least 42 killed in sectarian violence in Pakistan's Khyber Pakhtunkhwa\n* **Features**\n\t+ Ukraine war: Mapping the impact of climate change on global displacement\n\t+ Bollywood is turning to re-releases amid string of flops\n\t+ World reacts to ICC arrest warrants for Israel's Netanyahu, Gallant\n* **Video**\n\t+ Putin: Russia tested intermediate-range missile on Ukraine\n\t+ White House 'fundamentally rejects' ICC warrants for Israeli leaders\n\t+ Going bananas: Duct-taped fresh fruit sells for millions at Sotheby's\n\nNote: This summary only includes text that is relevant to news, announcements, and features, and ignores navigation-related links."

In [16]:
display(Markdown(context.message.content))

**Summary of Al Jazeera Website**

* **News and Announcements**
	+ Israeli air attack on Lebanon kills paramedics; ICC warrants issued for Netanyahu and Gallant
	+ Trump picks loyalist Pam Bondi as attorney general, Matt Gaetz withdraws
	+ Iran says activating 'advanced' centrifuges after IAEA censure
	+ At least 42 killed in sectarian violence in Pakistan's Khyber Pakhtunkhwa
* **Features**
	+ Ukraine war: Mapping the impact of climate change on global displacement
	+ Bollywood is turning to re-releases amid string of flops
	+ World reacts to ICC arrest warrants for Israel's Netanyahu, Gallant
* **Video**
	+ Putin: Russia tested intermediate-range missile on Ukraine
	+ White House 'fundamentally rejects' ICC warrants for Israeli leaders
	+ Going bananas: Duct-taped fresh fruit sells for millions at Sotheby's

Note: This summary only includes text that is relevant to news, announcements, and features, and ignores navigation-related links.